# 1. Tester et évaluer un modèle entraîné sur Google News

---

## 1.a


In [3]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: gensim in c:\python310\lib\site-packages (4.3.1)




[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import gensim.downloader as api
from gensim.models import KeyedVectors

In [3]:
# Pour obtenir le modèle depuis gensim
w2v_model = api.load("word2vec-google-news-300")


In [5]:
# Pour utiliser la copie locale du modèle
path_to_file = '~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz'
w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)

## 1.b

In [6]:
#%pip install psutil
import psutil
import os
memory_usage_bytes = psutil.Process(os.getpid()).memory_info().rss
memory_usage_mb = round(memory_usage_bytes / (1024 * 1024), 2)
print("The amount of memory occupied by the notebook process once the word vectors are loaded is:", memory_usage_mb, "MB")


The amount of memory occupied by the notebook process once the word vectors are loaded is: 19.6 MB


## 1.c

In [6]:
# Dimension de l'espace vectoriel dans lequel les mots sont représentés
print('Dimension :', w2v_vectors.vector_size)

Dimension : 300


## 1.d

In [7]:
# Taille du voc (nombre de mots différents) et 5 mots du voc
print('Taille du voc   :', len(w2v_vectors))
print('5 mots du voc   :', list(w2v_vectors.key_to_index.keys())[100:105])


Taille du voc   : 3000000
5 mots du voc   : ['company', 'any', 'team', 'against', 'off']


In [8]:
# Quelques mots hors vocabulaire. Test effectué en incluant les mots les plus rares en anglais
test_voc = ["Vexillology ","Quire","Clinomania","to","of"] 
for voc in test_voc:
    print(f"is the word {voc} in the vocabulary? {w2v_vectors.has_index_for(voc)}")

is the word Vexillology  in the vocabulary? False
is the word Quire in the vocabulary? True
is the word Clinomania in the vocabulary? False
is the word to in the vocabulary? False
is the word of in the vocabulary? False


## 1.e

In [10]:
# Distance entre les mots `rabbit` et `carrot`
round(w2v_vectors.similarity("rabbit", "carrot"), 2)

0.36

---

**Comment mesure-t-on ls distances entre deux mots dans cet espace ?**


Dans cet espace, on mesure les distances entre deux mot en utilisant la similarité cosinus, c'est-à-dire qu'on va observer l'angle formé entre les vecteurs représentant les mots. La similarité cosinus calcule le cosinus de cet angle ; ainsi, voici comment interpréter les résultats de cette métrique :     
*  1 : les deux vecteurs sont identiques ;
*  0 : les deux mots n'ont pas de rapport ;
* -1 : les deux mots sont opposés.

---

## 1.f

In [11]:
# Distance entre entre 5 paires de mots
words = ['wine', 'grapes', 'happy', 'sad', 'claws']

for i in range(5):
  w1 = words[i]
  w2 = words[(i+1)%4]
  print(w1, 'et', w2, ':', round(w2v_vectors.similarity(w1, w2), 2))

wine et grapes : 0.65
grapes et happy : 0.07
happy et sad : 0.54
sad et wine : 0.02
claws et grapes : 0.06


---

**Les distances obtenues correspondent-t-elles à vos intuitions sur la proximité des sens des mots ?**

Ces distances ne correspondent pas toutes à nos intuitions. Effectivement, bien que wine et grapes aient un score de similarity élevé comme on s'y attendait, on observe également une similarité positive plutôt élevée entre ``sad`` et ``happy``, alors qu'on aurait tendance à s'attendre à une valeur négative proche de -1 - de par leur sens opposé. 


 En réfléchissant un peu cependant, ça peut faire sens qu'ils soient proches étant donné que les deux expriment des émotions ; en ce sens là, ils sont proches. Aussi, word2vec est entraîné de sorte à valoriser la similarité des mots se retrouvant souvent ensemble dans les textes. Ce dernier argument nous indique encore une fois pourquoi notre intuition n'était pas correcte, ici.

---

## 1.g

**Pouvez-vous trouver des mots de sens opposés mais qui sont proches dans l’espace vectoriel ? Comment expliquez vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?**

Comme vu ci-dessus, les mots ``happy`` et ``sad`` sont considérés comme plutôt proches par le modèle word2vec pré-entraîné sur le corpus Google News, avec un score de 0.54. D'après le sens opposé des deux mots, on aurait tendance à supposer que leur score soit dans les négatifs, proche de -1. 

D'après nous, ceci s'explique premièrement par le fait que le modèle word2vec considère deux mots comme proches s'ils ont tendance à se retrouver dans les mêmes documents. Étant donné que le modèle qu'on utilise ici a été pré-entraîné sur le corpus Google News - qui comporte des articles en tous genres - le fait de trouver deux mots évoquant des émotions à sens opposé dans un même document n'est pas très surprenant ; ce qui aurait été le cas s'il s'agissait de textes de personnes évoquant leur vécu par rapport à une situation, par exemple. En somme, ces mots sont considérés comme proches certainement car ils expriment les deux des émotions, et sont donc similaires en cet aspect.

L'aspect de qualité ou de défaut est difficile à juger ici, étant donné que cette catégorisation dépendra de l'utilisation que l'on souhaite faire du modèle. Si l'on cherche à classifier des mots comme étant proches lorsqu'ils ont une dfinition, un sens profond, similaire et inversément, alors ce modèle n'est pas adapté.

## 1.h

In [16]:
# Calcul du score du modèle word2vec sur les données ``WordSimilarity-353``
from gensim.test.utils import datapath

similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print('Pearson  Result :\n statistic =', round(similarities[0][0],2), '\n pvalue =', similarities[0][1], '\n\n' +
      'Spearman Result :\n statistic =', round(similarities[1][0],2), '\n pvalue =', similarities[1][1], '\n\n' +
      'Percentage of test words not known to the model :\n', similarities[2])


Pearson  Result :
 statistic = 0.62 
 pvalue = 1.79632377241771e-39 

Spearman Result :
 statistic = 0.66 
 pvalue = 2.5346056459149263e-45 

Percentage of test words not known to the model :
 0.0


---

**Comment le score est-il calculé ? Que mesure-t-il ?**

Les scores fournis lorsqu'on évalue notre modèle sur les données de ce fichier sont les scores de Pearson et Spearman ; deux mesures nous fournissant un coefficient de corrélation entre les scores de similarité humaine et les scores de similarité prédits par le modèle. Ces deux coefficient sont compris entre -1 et 1, où -1 représente une corrélation négative parfaite, 0 représente aucune corrélation, et 1 représente une corrélation positive parfaite.

Le fichier `wordsim353.tsv` contient des paires de mots et leur score de similarité humaine - c'est-à-dire humainement attribué. Ainsi, ces deux scores nous permettent ici de mesurer la performance du modèle sur les données de `wordsim353.tsv`, selon deux types de corrélation.

Définissons maintenant ces deux coefficients : 

* <u>Pearson</u> : Évalue la relation linéaire entre les similarités observées par le modèle et les véridiques, c'est-à-dire le degré d'alignement du nuage de points formé par les résultats attendus et véridiques ;

* <u>Spearman</u> : Évalue à quel point la relation entre deux variables peut être décrite par une fonction monotone. Elle est étudiée lorsque deux variables statistiques semblent corrélées sans que la relation entre les deux variables soit de type affine. Elle consiste à trouver un coefficient de corrélation, non pas entre les valeurs prises par les deux variables mais entre les rangs de ces valeurs.

La p-value - quant à elle - mesure la probabilité que la corrélation observée soit due au hasard ou non. D'après nos recherches, une valeur faible - généralement définie comme inférieure à 0.05 - indique qu'il est peu probable que la statistique de corrélation observée ait été obtenue par hasard et que la corrélation est donc statistiquement significative. En revanche, une p-value élevée indique que la statistique de corrélation observée pourrait être due au hasard, et que la corrélation n'est donc pas statistiquement significative. 

Dans les deux cas ici, on observe une valeur très proche de zéro pour la p-value ; on en conclut ainsi que les coefficients de corrélation observés sont significatifs et donc recevables.

---

## 1.i

In [17]:
# Calcul du score du modèle word2vec sur les données ``questions-words.txt``
from gensim.test.utils import datapath

analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))


In [ ]:
round('Analogy score :', analogy_scores[0], 2)

0.74

---

**Comment le score est-il calculé ? Que mesure-t-il ?**

Ce score mesure la performance du modèle sur le set d'évaluation - `questions-words.txt` ici, il s'agit de l'équivalent de l'accuracy sur une tâche d'analogie de mots. L'évaluation d'un modèle Word2Vec sur une tâche d'analogie de mots permet de mesurer sa capacité à comprendre les relations sémantiques entre les mots, ce qui est une indication de sa qualité en tant que modèle de représentation de mots.

---
---

# 2. Entraîner deux nouveaux modèles word2vec à partir de nouveaux corpus

---

## 2.a

In [18]:
# Récupération du corpus contenant les 10^8 premiers caractères de Wikipédia (en anglais)
import gensim.downloader as api

corpus = api.load('text8')

[==================================================] 100.0% 31.6/31.6MB downloaded


In [21]:
api.info('text8')

{'num_records': 1701,
 'record_format': 'list of str (tokens)',
 'file_size': 33182058,
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'license': 'not found',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.',
 'checksum': '68799af40b6bda07dfa47a32612e5364',
 'file_name': 'text8.gz',
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'parts': 1}

In [22]:
# Nombre de phrases du corpus 
print('Number of sentences in the corpus :', api.info('text8')['num_records'])

Number of sentences in the corpus : 1701


In [23]:
# Nombre de mots (token) du corpus

print('Number of tokens in the corpus :', sum(len(token) for token in corpus))


Number of tokens in the corpus : 17005207


## 2.b

In [20]:
# Entraînement d'un nouveau modèle word2vec sur ce nouveau corpus
from gensim.models import Word2Vec

model = Word2Vec(corpus, vector_size=300)

In [24]:
w2v_vectors = model.wv

print('Dimension choosed for the new model\'s embedding :', w2v_vectors.vector_size)

Dimension choosed for the new model's embedding : 300


---

**Remarque :**

*Le choix de la dimension choisie pour l'embedding de ce modèle a été fait de sorte à choisir la même dimension que ceux du modèle word2vec pré-entraîné sur les données Google News, ceci afin de permettre une comparaison sur une même base vectorielle.*

**• Combien de temps prend l’entraînement sur le corpus total ?**

1m 27s.

---

In [27]:
# Taille (en Mo) du modèle word2vec résultant 
import sys

model.save('word2vec_text8.model')

print('The model size is 2.08 Mo.')

The model size is 2.08 Mo.


## 2.c

In [28]:
# Mesure de la qualité de ce modèle comme dans la partie 1 point i

# Calcul du score du modèle sur les données ``WordSimilarity-353``
from gensim.test.utils import datapath

similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print('Pearson  Result :\n statistic =', round(similarities[0][0],2), '\n pvalue =', similarities[0][1], '\n\n' +
      'Spearman Result :\n statistic =', round(similarities[1][0],2), '\n pvalue =', similarities[1][1], '\n\n' +
      'Ratio of pairs with unknown words :\n', similarities[2])

Pearson  Result :
 statistic = 0.61 
 pvalue = 6.883556714314934e-37 

Spearman Result :
 statistic = 0.63 
 pvalue = 1.7145521521580297e-39 

Ratio of pairs with unknown words :
 0.56657223796034


In [31]:
# Mesure de la qualité de ce modèle comme dans la partie 1 point j

# Calcul du score du modèle word2vec sur les données ``questions-words.txt``
from gensim.test.utils import datapath

analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))



In [36]:
print('Analogy score :', analogy_scores[0])

Analogy score : 0.25775509059292084


---

**Ce modèle est-il meilleur que celui entraîné sur Google News ? Quelle serait la raison de la différence ?**

Tableau répicatulatif des scores :

|                 | Pearson | Spearman | Analogy |
| --------------- | ------- | -------- | ------- |
| **text8**       | 0.61    | 0.63     | 0.25    |
| **Google News** | 0.62    | 0.66     | 0.74    |



ICI !!

* Modèle entraîné sur Google News : Ces deux valeurs sont ici très proches (`Pearson : 62%`, `Spearman : 66%`) l'une de l'autre. On peut donc en conclure que les résultats du modèles sont plutôt corrélés avec les résultats attendus ; ces scores étant plus élevés que 50%. La corrélation n'est cependant pas très élevée, aux alentours de 60% ; ce qui nous fait penser que l'accuracy sera bonne mais pas notablement non plus.

* 

---

## 2.d

In [46]:
# Téléchargement du corpus quatre fois plus grand constitué de la concaténation du corpus text8
# et des dépêches économiques de Reuters (413 Mo)
from gensim.models.word2vec import Text8Corpus
wiki_aug_model = Word2Vec(Text8Corpus('wikipedia_augmented.dat'), vector_size=300)
wiki_aug_model.save("wikipedia_augmented.model")


In [49]:
w2v_vectors = wiki_aug_model.wv


La dimension choisie pour l'embedding de ce modèle reste identiques à celle des modèles précédents, c-à-d 300.

In [53]:
# Nombre de mots (token) du corpus
corpus = Text8Corpus('wikipedia_augmented.dat')
print('Number of tokens in the corpus :', sum(len(token) for token in corpus))

Number of tokens in the corpus : 70102071


In [57]:
# La différence au niveau des tokens des différents corpus 
text8_token = 17005207 
wikipedia_augmented_token=70102071
diff = wikipedia_augmented_token - text8_token
print("Difference between text8 and widipedia augmented token is : ",diff)


Difference between text8 and widipedia augmented token is :  53096864


---

**• Combien de temps prend l’entraînement ?**

Environ 6min 50s

**• Quelle est la taille (en Mo) du modèle word2vec résultant ?**

3.71Mo

---

## 2.e

In [50]:
# Mesure de la qualité de ce modèle comme dans la partie 1 point i

# Calcul du score du modèle sur les données ``WordSimilarity-353``
from gensim.test.utils import datapath

similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print('Pearson  Result :\n statistic =', round(similarities[0][0],2), '\n pvalue =', similarities[0][1], '\n\n' +
      'Spearman Result :\n statistic =', round(similarities[1][0],2), '\n pvalue =', similarities[1][1], '\n\n' +
      'Ratio of pairs with unknown words :\n', similarities[2])

Pearson  Result :
 statistic = 0.5 
 pvalue = 6.3654283708341675e-24 

Spearman Result :
 statistic = 0.51 
 pvalue = 2.7513109548527965e-25 

Ratio of pairs with unknown words :
 0.0


In [51]:
# Mesure de la qualité de ce modèle comme dans la partie 1 point j

# Calcul du score du modèle word2vec sur les données ``questions-words.txt``
from gensim.test.utils import datapath

analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'))

analogy_scores[0]

0.35773330432891015

---

**Est-il meilleur que le précédent ? Pour quelle raison ?**

wesh

---


Tableau répicatulatif des scores :

|                         | Pearson | Spearman | Analogy |
| ----------------------- | ------- | -------- | ------- |
| **text8**               | 0.61    | 0.63     | 0.25    |
| **Google News**         | 0.62    | 0.66     | 0.74    |
| **Wikipedia Augmented** | 0.5     | 0.51     | 0.35    |

---


Fin du labo.